In [ ]:
# Exercício 2 -> Previsão de tempo de resfriamento

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1. Criar dataset simulado
np.random.seed(43)
n = 100

dados_resfriamento = pd.DataFrame({
    "temperatura_molde": np.random.uniform(150, 300, n),
    "pressao_injecao": np.random.uniform(500, 1500, n),
    "tipo_polimero": np.random.choice(["PEAD", "PP", "PVC"], n, p=[0.4, 0.3, 0.3])
})

# Codificação temporária para simular a variável-alvo
polimero_dummies = pd.get_dummies(dados_resfriamento['tipo_polimero'], prefix='polimero', drop_first=True)

# Criar a variável-alvo (tempo_resfriamento)
# Tempo aumenta com a temperatura (principalmente), e varia pelo polímero (PVC>PP>PEAD)
dados_resfriamento["tempo_resfriamento"] = (
    0.15 * dados_resfriamento["temperatura_molde"] + # Relação principal e quase linear
    0.005 * dados_resfriamento["pressao_injecao"] + # Impacto menor
    8 * polimero_dummies.get("polimero_PVC", 0) + # PVC (maior tempo)
    3 * polimero_dummies.get("polimero_PP", 0) +  # PP (tempo intermediário)
    # PEAD é o baseline (menor tempo)
    np.random.normal(0, 2.0, n) # Ruído
)

# 2. Pré-processar os dados
# Codificação One-Hot para 'tipo_polimero'
dados_resfriamento = pd.get_dummies(dados_resfriamento, columns=['tipo_polimero'], drop_first=True)

# Variáveis Preditivas (X) e Variável Alvo (y)
X_r = dados_resfriamento.drop("tempo_resfriamento", axis=1)
y_r = dados_resfriamento["tempo_resfriamento"]

# 3. Dividir os dados em treino e teste
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_r, y_r, test_size=0.3, random_state=43)

# Modelos a comparar
modelos_r = {
    "Regressão Linear": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=43)
}

resultados_resfriamento = {}

# 4. Treinar e avaliar os modelos
print("Exercício 2: Previsão do Tempo de Resfriamento\n")
for nome, modelo in modelos_r.items():
    modelo.fit(X_train_r, y_train_r)
    y_pred_r = modelo.predict(X_test_r)

    # Cálculo das métricas
    mse = mean_squared_error(y_test_r, y_pred_r)
    mae = mean_absolute_error(y_test_r, y_pred_r)
    r2 = r2_score(y_test_r, y_pred_r)

    resultados_resfriamento[nome] = {"EQM": mse, "EAM": mae, "R2": r2}

# 5. Exibir as métricas
df_resultados_resfriamento = pd.DataFrame(resultados_resfriamento).T

print(df_resultados_resfriamento.sort_values(by='R2', ascending=False))
print("\nO modelo Random Forest obteve um R2 ligeiramente melhor (maior) e menores EQM/EAM.")